## Create “subset” files for:

a) Radars -  6 degree by 6 degree box centered on the radar
- Cedar City, Utah - 37 degrees 35 minutes 27 seconds; 112 51 44
- Phoenix, AZ - 33 17 21 ; 111 40 12
- Greer, SC - 34 53 00 ; 082 13 12
- Sterling, VA - 38 58 31 ; 077 28 40 
- Denver, CO - 39 47 12 ; 104 32 45

b) Columbia Plateau and Surroundings: 41 - 49 lat by 123 - 115 lon

In [1]:
import os
import numpy as np
import pandas as pd
import xarray as xr

In [2]:
from __init__ import *

In [3]:
pd.DataFrame(cities).T

,lat,lon,path,r
cedar,37.5908,-112.862,/home/jsignell/erddapData/Cloud_to_Ground_Ligh...,3
columbia,45,-119,/home/jsignell/erddapData/Cloud_to_Ground_Ligh...,4
denver,39.7867,-104.546,/home/jsignell/erddapData/Cloud_to_Ground_Ligh...,3
greer,34.8833,-82.22,/home/jsignell/erddapData/Cloud_to_Ground_Ligh...,3
phoenix,33.2892,-111.67,/home/jsignell/erddapData/Cloud_to_Ground_Ligh...,3
sterling,38.9753,-77.4778,/home/jsignell/erddapData/Cloud_to_Ground_Ligh...,3


Benchmarking outcomes:
 - 2015_07_14 (2808266 records): 7.63ms to open dataset, 951ms to bound, save, and close
 - 1991_01_22 (2 records): 2.25ms to open dataset, 8.48ms to bound, and close

In [4]:
def bound(ds, lat, lon, r):
    bounding_box = ((ds.lat<(lat+r)) & (ds.lat>(lat-r)) & (ds.lon<(lon+r)) & (ds.lon>(lon-r)))
    return(ds.where(bounding_box).dropna('record'))

In [5]:
fnames = os.listdir(out_path)

In [21]:
attrs = {'semimajor': {'long_name': 'Semimajor Axis of 50% probability ellipse for each flash',
                       'units': 'km'},
         'semiminor': {'long_name': 'Semiminor Axis of 50% probability ellipse for each flash',
                       'units': 'km'},
         'ratio': {'long_name': 'Ratio of Semimajor to Semiminor'},
         'angle': {'long_name': 'Angle of 50% probability ellipse from North',
                   'units': 'Deg'},
         'chi_squared': {'long_name': 'Chi-squared value of statistical calculation'},
         'nsensors': {'long_name': 'Number of sensors reporting the flash'},
         'cloud_ground': {'long_name': 'Cloud_to_Ground or In_Cloud Discriminator'},
         'amplitude': {'units': 'kA',
                       'long_name': 'Polarity and strength of strike'},
         'strokes': {'long_name': 'multiplicity of flash'}}

In [28]:
path = cities['sterling']['path']
for fname in os.listdir(path):
    ds0 = xr.open_dataset(path+fname)
    vars = ds0.data_vars.keys()
    for var in vars:
        ds0[var].attrs.update(attrs[var])
    vars.extend(ds0.coords.keys())
    for var in vars:
        if var == 'cloud_ground':
            ds0[var].encoding.update({'dtype': 'S1'})
        elif var == 'nsensors' or var == 'strokes':
            ds0[var].encoding.update({'dtype': np.int32})
        else:
            ds0[var].encoding.update({'dtype': np.double})
        
    ds0.to_netcdf('{path}{fname}'.format(path=out+'test1/', fname=fname), 
                  format='netCDF4', engine='netcdf4')

In [25]:
count=0
for fname in fnames:
    ds = xr.open_dataset(out_path+fname)
    for k, v in cities.items():
        ds0 = bound(ds, v['lat'],v['lon'],v['r'])
        if ds0.dims.values()[0] > 0:
        vars = ds0.data_vars.keys()
        for var in vars:
            ds0[var].attrs.update(attrs[var])
        vars.extend(ds0.coords.keys())
        for var in vars:
            if var == 'cloud_ground':
                ds0[var].encoding.update({'dtype': 'S1'})
            elif var == 'nsensors' or var == 'strokes':
                ds0[var].encoding.update({'dtype': np.int32})
            else:
                ds0[var].encoding.update({'dtype': np.double})

        ds0.to_netcdf('{path}{fname}'.format(path=v['path'], fname=fname), 
                      format='netCDF4', engine='netcdf4')
        ds0.close()
    ds.close()
    count+=1
    if count%10 == 0:
        print fname

1995_01_10.nc
1995_01_20.nc
1995_01_30.nc
1995_02_09.nc
1995_02_19.nc
1995_03_01.nc
1995_03_11.nc
1995_03_21.nc
1995_03_31.nc
1995_04_10.nc
1995_04_20.nc
1995_04_30.nc
1995_05_10.nc
1995_05_20.nc
1995_05_30.nc
1995_06_09.nc
1995_06_19.nc
1995_06_29.nc
1995_07_09.nc
1995_07_19.nc
1995_07_29.nc
1995_08_08.nc
1995_08_18.nc
1995_08_28.nc
1995_09_07.nc
1995_09_17.nc
1995_09_27.nc
1995_10_07.nc
1995_10_17.nc
1995_10_27.nc
1995_11_06.nc
1995_11_16.nc
1995_11_26.nc
1995_12_06.nc
1995_12_16.nc
1995_12_26.nc
1996_01_05.nc
1996_01_15.nc
1996_01_25.nc
1996_02_04.nc
1996_02_14.nc
1996_02_24.nc
1996_03_05.nc
1996_03_15.nc
1996_03_25.nc
1996_04_04.nc
1996_04_14.nc
1996_04_24.nc
1996_05_04.nc
1996_05_14.nc
1996_05_24.nc
1996_06_03.nc
1996_06_13.nc
1996_06_23.nc
1996_07_03.nc
1996_07_13.nc
1996_07_23.nc
1996_08_02.nc
1996_08_12.nc
1996_08_22.nc
1996_09_01.nc
1996_09_11.nc
1996_09_21.nc
1996_10_01.nc
1996_10_11.nc
1996_10_21.nc
1996_10_31.nc
1996_11_10.nc
1996_11_20.nc
1996_11_30.nc
1996_12_10.nc
1996_1